In [6]:
import pandas as pd
import numpy as np
import requests
import os
import json

In [33]:
politicians = pd.read_csv('politicians.csv')
politicians = politicians[(politicians['Political_party'] == 'Democratic Party') | (politicians['Political_party'] == 'Republican Party')]
tweets = pd.DataFrame(columns = ['id', 'text','party'])
bearer_token = os.environ.get('BEARER_TOKEN') #can do export'BEARER_TOKEN'=$your_token
                                              #not showing my token here for what I hope are obvious reasons
#huge antipattern but I couldn't figure out a better way to do this
for accid, party in zip(politicians['Account_ID'], politicians['Political_party']):
    temp = fill_tweets(bearer_token,accid, party)
    if (temp.empty):
        print('empty!')
    else:
        tweets = tweets.append(temp)

In [31]:
def create_url(accid):
    query = "from:{}".format(accid)
    fields = "tweet.fields=text"
    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}".format(
    query, fields
    )
    return url
    
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [32]:
def fill_tweets(token, accid, party):
    url = create_url(accid)
    headers = create_headers(token)
    json_response = connect_to_endpoint(url, headers)
    if 'data' in json_response:
        app = pd.DataFrame.from_records(json_response['data'])
        app['party'] = party
    else:
        return pd.DataFrame(columns = ['id', 'text', 'party'])
    return app

In [36]:
tweets.shape

(1355, 3)

In [37]:
tweets.to_pickle('tweets.pkl')